In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

#Initialize data source and conditions
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
parameters = {
  'start':'1',
  'limit':'20',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
}

#Initialize session
session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
type(data)

In [ ]:
import pandas as pd


#Displays all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#Normalize data
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

In [ ]:
#Define the api_runner function for automation
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'20',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    # Using method via dataframes
    df2 = pd.json_normalize(data['data'])
    df2['timestamp'] = pd.to_datetime('now')
    df = df.append(df2)


    # Using method via CSV (specify the filepath)
    #df = pd.json_normalize(data['data'])
    #df['timestamp'] = pd.to_datetime('now')
    #df

    #if not os.path.isfile(r'C:\Users.csv'):
        #df.to_csv(r'C:\Users...csv', header='column_names')
    #else:
        #df.to_csv(r'C:\Users...csv', mode='a', header=False)
        
    #Then to read in the file: df = pd.read_csv(r'C:\Users...csv')

In [ ]:
#Conduct the automatic scrape of Crypto API
import os 
from time import time
from time import sleep

for i in range(10): #Number of pulls
    api_runner()
    print('API Runner completed')
    sleep(60) #sleep for 1 minute (60 secs)

In [ ]:
#If CSV is used
#df1a = pd.read_csv(r'C:\Users...csv')
#df1a

In [ ]:
df

In [ ]:
# Set decimal placement

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df

In [ ]:
# Looking at coin trends over time

df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df3

In [ ]:
# Looking at coin trends over time

df3a = df.groupby('name', sort=False, as_index = False)[['num_market_pairs']].mean()
df3a = df3a.head(7)
df3a

In [ ]:
df4 = df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
df5 = df4.to_frame(name='values')
df5

In [ ]:
#Will use this number for the range in next step
df5.count()

In [ ]:
#Create the range for the dataframe
index = pd.Index(range(120))

# Set the above DataFrame index object as the index
# using set_index()/reset_index() function
df6 = df5.reset_index()
df6

# If it only has the index and values try doing reset_index like "df5.reset_index()"

In [ ]:
# Change the column name
df7 = df6.rename(columns={'level_1': 'percent_change'})
df7

In [ ]:
#Rename values
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d','quote.USD.percent_change_1h'],['24h','7d','30d','60d','90d','1h'])
df7

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# All cryptocurrencies and their performance
sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')

In [ ]:
# Time series of single cryptocurrency

df10 = df[['name','quote.USD.price','timestamp']]
df10 = df10.query("name == 'Bitcoin'")
df10

df11 = df[['name','quote.USD.price','timestamp']]
df11 = df11.query("name == 'Ethereum'")
df11

In [ ]:
#10 minute (or custom range) series of Crypto Price
sns.set_theme(style="darkgrid")

sns.lineplot(x='timestamp', y='quote.USD.price', data = df10)

In [ ]:
#10 minute (or custom range) series of Crypto Price
sns.set_theme(style="darkgrid")

sns.lineplot(x='timestamp', y='quote.USD.price', data = df11)

In [ ]:
#Number of Market Pairs for Top 7 Cryptocurrencies
sns.set_theme(style="darkgrid")

sns.barplot(x='name', y='num_market_pairs', data = df3a)